##### Copyright 2021 The IREE Authors

In [1]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# Variables and State

This notebook

1. Creates a TensorFlow program with basic tf.Variable use
2. Imports that program into IREE's compiler
3. Compiles the imported program to an IREE VM bytecode module
4. Tests running the compiled VM module using IREE's runtime
5. Downloads compilation artifacts for use with the native (C API) sample application

In [2]:
#@title General setup

import os
import tempfile

ARTIFACTS_DIR = os.path.join(tempfile.gettempdir(), "iree", "colab_artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
print(f"Using artifacts directory '{ARTIFACTS_DIR}'")

Using artifacts directory '/tmp/iree/colab_artifacts'


## Create a program using TensorFlow and import it into IREE

This program uses `tf.Variable` to track state internal to the program then exports functions which can be used to interact with that variable.

Note that each function we want to be callable from our compiled program needs
to use `@tf.function` with an `input_signature` specified.

References:

* ["Introduction to Variables" Guide](https://www.tensorflow.org/guide/variable)
* [`tf.Variable` reference](https://www.tensorflow.org/api_docs/python/tf/Variable)
* [`tf.function` reference](https://www.tensorflow.org/api_docs/python/tf/function)

In [3]:
#@title Define a simple "counter" TensorFlow module

import tensorflow as tf

class CounterModule(tf.Module):
  def __init__(self):
    super().__init__()
    self.counter = tf.Variable(0)

  @tf.function(input_signature=[])
  def get_value(self):
    return self.counter
    
  @tf.function(input_signature=[tf.TensorSpec([], tf.int32)])
  def set_value(self, new_value):
    self.counter.assign(new_value)
    
  @tf.function(input_signature=[tf.TensorSpec([], tf.int32)])
  def add_to_value(self, x):
    self.counter.assign(self.counter + x)

  @tf.function(input_signature=[])
  def reset_value(self):
    self.set_value(0)

In [4]:
%%capture
!python -m pip install iree-compiler iree-tools-tf -f https://github.com/google/iree/releases

In [5]:
#@title Import the TensorFlow program into IREE as MLIR

from IPython.display import clear_output

from iree.compiler import tf as tfc

compiler_module = tfc.compile_module(
    CounterModule(), import_only=True, output_mlir_debuginfo=False)
clear_output()  # Skip over TensorFlow's output.

# Print the imported MLIR to see how the compiler views this TensorFlow program.
# Note IREE's `util.global` ops and the public (exported) functions.
print("Counter MLIR:\n```\n%s```\n" % compiler_module.decode("utf-8"))

# Save the imported MLIR to disk.
imported_mlir_path = os.path.join(ARTIFACTS_DIR, "counter.mlir")
with open(imported_mlir_path, "wt") as output_file:
  output_file.write(compiler_module.decode("utf-8"))
print(f"Wrote MLIR to path '{imported_mlir_path}'")

Counter MLIR:
```
"builtin.module"() ( {
  "iree_input.global"() {initial_value = dense<0> : tensor<i32>, is_mutable, sym_name = "counter", sym_visibility = "private", type = tensor<i32>} : () -> ()
  "builtin.func"() ( {
  ^bb0(%arg0: !iree_input.buffer_view):  // no predecessors
    %0 = "iree_input.cast.buffer_view_to_tensor"(%arg0) : (!iree_input.buffer_view) -> tensor<i32>
    "std.call"(%0) {callee = @__inference_add_to_value_100} : (tensor<i32>) -> ()
    "std.return"() : () -> ()
  }) {iree.abi = "{\22a\22:[[\22ndarray\22,\22i32\22,0]],\22r\22:[],\22v\22:1}", sym_name = "add_to_value", type = (!iree_input.buffer_view) -> ()} : () -> ()
  "builtin.func"() ( {
  ^bb0(%arg0: tensor<i32>):  // no predecessors
    %0 = "iree_input.global.address"() {global = @counter} : () -> !iree_input.ptr<tensor<i32>>
    %1 = "iree_input.global.load.indirect"(%0) : (!iree_input.ptr<tensor<i32>>) -> tensor<i32>
    %2 = "chlo.broadcast_add"(%1, %arg0) : (tensor<i32>, tensor<i32>) -> tensor<i32>
 

## Test the imported program

_Note: you can stop after each step and use intermediate outputs with other tools outside of Colab._

_See the [README](https://github.com/google/iree/tree/main/iree/samples/variables_and_state#changing-compilation-options) for more details and example command line instructions._

* _The "imported MLIR" can be used by IREE's generic compiler tools_
* _The "flatbuffer blob" can be saved and used by runtime applications_

_The specific point at which you switch from Python to native tools will depend on your project._

In [6]:
%%capture
!python -m pip install iree-compiler -f https://github.com/google/iree/releases

In [7]:
#@title Compile the imported MLIR further into an IREE VM bytecode module

from iree.compiler import compile_str

flatbuffer_blob = compile_str(compiler_module, target_backends=["vmvx"], input_type="mhlo")

# Save the imported MLIR to disk.
flatbuffer_path = os.path.join(ARTIFACTS_DIR, "counter_vmvx.vmfb")
with open(flatbuffer_path, "wb") as output_file:
  output_file.write(flatbuffer_blob)
print(f"Wrote .vmfb to path '{flatbuffer_path}'")

Wrote .vmfb to path '/tmp/iree/colab_artifacts/counter_vmvx.vmfb'


In [8]:
%%capture
!python -m pip install iree-runtime -f https://github.com/google/iree/releases

In [9]:
#@title Test running the compiled VM module using IREE's runtime

from iree import runtime as ireert

vm_module = ireert.VmModule.from_flatbuffer(flatbuffer_blob)
config = ireert.Config("vmvx")
ctx = ireert.SystemContext(config=config)
ctx.add_vm_module(vm_module)

In [10]:
# Our @tf.functions are accessible by name on the module named 'module'
counter = ctx.modules.module

print(counter.get_value())
counter.set_value(101)
print(counter.get_value())

counter.add_to_value(20)
print(counter.get_value())
counter.add_to_value(-50)
print(counter.get_value())

counter.reset_value()
print(counter.get_value())

0
101
121
71
0


## Download compilation artifacts

In [11]:
ARTIFACTS_ZIP = "/tmp/variables_and_state_colab_artifacts.zip"

print(f"Zipping '{ARTIFACTS_DIR}' to '{ARTIFACTS_ZIP}' for download...")
!cd {ARTIFACTS_DIR} && zip -r {ARTIFACTS_ZIP} .

# Note: you can also download files using Colab's file explorer
try:
  from google.colab import files
  print("Downloading the artifacts zip file...")
  files.download(ARTIFACTS_ZIP)
except ImportError:
  print("Missing google_colab Python package, can't download files")

Zipping '/tmp/iree/colab_artifacts' to '/tmp/variables_and_state_colab_artifacts.zip' for download...
  adding: counter_vmvx.vmfb (deflated 61%)
  adding: counter.mlir (deflated 82%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>